# Agent Memory: Building Memory-Enabled Agents with LangGraph

In this notebook, we'll explore **agent memory systems** - the ability for AI agents to remember information across interactions. We'll implement all five memory types from the **CoALA (Cognitive Architectures for Language Agents)** framework while building on our Personal Wellness Assistant use case.

**Learning Objectives:**
- Understand the 5 memory types from the CoALA framework
- Implement short-term memory with checkpointers and thread_id
- Build long-term memory with InMemoryStore and namespaces
- Use semantic memory for meaning-based retrieval
- Apply episodic memory for few-shot learning from past experiences
- Create procedural memory for self-improving agents
- Combine all memory types into a unified wellness agent

## Table of Contents:

- **Breakout Room #1:** Memory Foundations
  - Task 1: Dependencies
  - Task 2: Understanding Agent Memory (CoALA Framework)
  - Task 3: Short-Term Memory (MemorySaver, thread_id)
  - Task 4: Long-Term Memory (InMemoryStore, namespaces)
  - Task 5: Message Trimming & Context Management
  - Question #1 & Question #2
  - 🏗️ Activity #1: Store & Retrieve User Wellness Profile

- **Breakout Room #2:** Advanced Memory & Integration
  - Task 6: Semantic Memory (Embeddings + Search)
  - Task 7: Building Semantic Wellness Knowledge Base
  - Task 8: Episodic Memory (Few-Shot Learning)
  - Task 9: Procedural Memory (Self-Improving Agent)
  - Task 10: Unified Wellness Memory Agent
  - Question #3 & Question #4
  - 🏗️ Activity #2: Wellness Memory Dashboard

---
# 🤝 Breakout Room #1
## Memory Foundations

## Task 1: Dependencies

Before we begin, make sure you have:

1. **API Keys** for:
   - OpenAI (for GPT-4o-mini and embeddings)
   - LangSmith (optional, for tracing)

2. **Dependencies installed** via `uv sync`

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)


True

In [2]:
import os
print("ENDPOINT:", os.getenv("LANGCHAIN_ENDPOINT"))
print("ORG:", os.getenv("LANGCHAIN_ORG_ID"))
print("PROJECT:", os.getenv("LANGCHAIN_PROJECT"))


ENDPOINT: https://eu.api.smith.langchain.com
ORG: 635a6074-4bcb-4dc8-919d-2a474203c6d7
PROJECT: 06_Agent_Memory


In [3]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

In [4]:
# Set API Keys
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

In [5]:
# Optional: LangSmith for tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Agent Memory - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key (press Enter to skip): ") or ""

if not os.environ["LANGCHAIN_API_KEY"]:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")
else:
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")

LangSmith tracing enabled. Project: AIE9 - Agent Memory - e11e7eda


In [6]:
# Initialize LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Test the connection
response = llm.invoke("Say 'Memory systems ready!' in exactly those words.")
print(response.content)

Memory systems ready!


## Task 2: Understanding Agent Memory (CoALA Framework)

The **CoALA (Cognitive Architectures for Language Agents)** framework identifies 5 types of memory that agents can use:

| Memory Type | Human Analogy | AI Implementation | Wellness Example |
|-------------|---------------|-------------------|------------------|
| **Short-term** | What someone just said | Conversation history within a thread | Current consultation conversation |
| **Long-term** | Remembering a friend's birthday | User preferences stored across sessions | User's goals, allergies, conditions |
| **Semantic** | Knowing Paris is in France | Facts retrieved by meaning | Wellness knowledge retrieval |
| **Episodic** | Remembering your first day at work | Learning from past experiences | Past successful advice patterns |
| **Procedural** | Knowing how to ride a bike | Self-improving instructions | Learned communication preferences |

### Memory Architecture Overview

```
┌─────────────────────────────────────────────────────────────────┐
│                    LangGraph Wellness Agent                     │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐           │
│  │  Short-term  │  │  Long-term   │  │   Semantic   │           │
│  │    Memory    │  │    Memory    │  │    Memory    │           │
│  │              │  │              │  │              │           │
│  │ Checkpointer │  │    Store     │  │Store+Embed   │           │
│  │ + thread_id  │  │ + namespace  │  │  + search()  │           │
│  └──────────────┘  └──────────────┘  └──────────────┘           │
│                                                                 │
│  ┌──────────────┐  ┌──────────────┐                             │
│  │   Episodic   │  │  Procedural  │                             │
│  │    Memory    │  │    Memory    │                             │
│  │              │  │              │                             │
│  │  Few-shot    │  │Self-modifying│                             │
│  │  examples    │  │   prompts    │                             │
│  └──────────────┘  └──────────────┘                             │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Key LangGraph Components

| Component | Memory Type | Scope |
|-----------|-------------|-------|
| `MemorySaver` (Checkpointer) | Short-term | Within a single thread |
| `InMemoryStore` | Long-term, Semantic, Episodic, Procedural | Across all threads |
| `thread_id` | Short-term | Identifies unique conversations |
| Namespaces | All store-based | Organizes memories by user/purpose |

**Documentation:**
- [CoALA Paper](https://arxiv.org/abs/2309.02427)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)

## Task 3: Short-Term Memory (MemorySaver, thread_id)

**Short-term memory** maintains context within a single conversation thread. Think of it like your working memory during a phone call - you remember what was said earlier, but once the call ends, those details fade.

In LangGraph, short-term memory is implemented through:
- **Checkpointer**: Saves the graph state at each step
- **thread_id**: Uniquely identifies each conversation

### How It Works

```
Thread 1: "Hi, I'm Alice"          Thread 2: "What's my name?"
     │                                   │
     ▼                                   ▼
┌──────────────┐                   ┌──────────────┐
│ Checkpointer │                   │ Checkpointer │
│  thread_1    │                   │  thread_2    │
│              │                   │              │
│ ["Hi Alice"] │                   │ [empty]      │
└──────────────┘                   └──────────────┘
     │                                   │
     ▼                                   ▼
"Hi Alice!"                        "I don't know your name"
```

In [7]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Define the state schema for our graph
# The `add_messages` annotation tells LangGraph how to update the messages list
class State(TypedDict):
    messages: Annotated[list, add_messages]


# Define our wellness chatbot node
def wellness_chatbot(state: State):
    """Process the conversation and generate a wellness-focused response."""
    system_prompt = SystemMessage(content="""You are a friendly Personal Wellness Assistant. 
Help users with exercise, nutrition, sleep, and stress management questions.
Be supportive and remember details the user shares about themselves.""")
    
    messages = [system_prompt] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build the graph
builder = StateGraph(State)
builder.add_node("chatbot", wellness_chatbot)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

# Compile with a checkpointer for short-term memory
checkpointer = MemorySaver()
wellness_graph = builder.compile(checkpointer=checkpointer)

print("Wellness chatbot compiled with short-term memory (checkpointing)")

Wellness chatbot compiled with short-term memory (checkpointing)


In [8]:
# Test short-term memory within a thread
config = {"configurable": {"thread_id": "wellness_thread_1"}}

# First message - introduce ourselves
response = wellness_graph.invoke(
    {"messages": [HumanMessage(content="Hi! My name is Sarah and I want to improve my sleep.")]},
    config
)
print("User: Hi! My name is Sarah and I want to improve my sleep.")
print(f"Assistant: {response['messages'][-1].content}")
print()

User: Hi! My name is Sarah and I want to improve my sleep.
Assistant: Hi Sarah! It's great to meet you! Improving sleep is such an important goal for overall wellness. Can you tell me a bit more about your current sleep habits? For example, how many hours do you usually sleep, and do you have any specific challenges that keep you from getting restful sleep?



In [9]:
# Second message - test if it remembers (same thread)
response = wellness_graph.invoke(
    {"messages": [HumanMessage(content="What's my name and what am I trying to improve?")]},
    config  # Same config = same thread_id
)
print("User: What's my name and what am I trying to improve?")
print(f"Assistant: {response['messages'][-1].content}")

User: What's my name and what am I trying to improve?
Assistant: Your name is Sarah, and you're looking to improve your sleep. If you have any specific challenges or habits you'd like to share, I'm here to help!


In [10]:
# New thread - it won't remember Sarah!
different_config = {"configurable": {"thread_id": "wellness_thread_2"}}

response = wellness_graph.invoke(
    {"messages": [HumanMessage(content="What's my name?")]},
    different_config  # Different thread_id = no memory of Sarah
)
print("User (NEW thread): What's my name?")
print(f"Assistant: {response['messages'][-1].content}")
print()
print("Notice: The agent doesn't know our name because this is a new thread!")

User (NEW thread): What's my name?
Assistant: I don't have your name yet! If you'd like to share it, I can remember it for our future conversations. How can I assist you today?

Notice: The agent doesn't know our name because this is a new thread!


In [11]:
# Inspect the state of thread 1
state = wellness_graph.get_state(config)
print(f"Thread 1 has {len(state.values['messages'])} messages:")
for msg in state.values['messages']:
    role = "User" if isinstance(msg, HumanMessage) else "Assistant"
    content = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
    print(f"  {role}: {content}")

Thread 1 has 4 messages:
  User: Hi! My name is Sarah and I want to improve my sleep.
  Assistant: Hi Sarah! It's great to meet you! Improving sleep is such an important goal for ...
  User: What's my name and what am I trying to improve?
  Assistant: Your name is Sarah, and you're looking to improve your sleep. If you have any sp...


## Task 4: Long-Term Memory (InMemoryStore, namespaces)

**Long-term memory** stores information across different conversation threads. This is like remembering that your friend prefers tea over coffee - you remember it every time you meet them, regardless of what you're currently discussing.

In LangGraph, long-term memory uses:
- **Store**: A persistent key-value store
- **Namespaces**: Organize memories by user, application, or context

### Key Difference from Short-Term Memory

| Short-Term (Checkpointer) | Long-Term (Store) |
|---------------------------|-------------------|
| Scoped to a single thread | Shared across all threads |
| Automatic (messages) | Explicit (you decide what to store) |
| Conversation history | User preferences, facts, etc. |

In [12]:
from langgraph.store.memory import InMemoryStore

# Create a store for long-term memory
store = InMemoryStore()

# Namespaces organize memories - typically by user_id and category
user_id = "user_sarah"
profile_namespace = (user_id, "profile")
preferences_namespace = (user_id, "preferences")

# Store Sarah's wellness profile
store.put(profile_namespace, "name", {"value": "Sarah"})
store.put(profile_namespace, "goals", {"primary": "improve sleep", "secondary": "reduce stress"})
store.put(profile_namespace, "conditions", {"allergies": ["peanuts"], "injuries": ["bad knee"]})

# Store Sarah's preferences
store.put(preferences_namespace, "communication", {"style": "friendly", "detail_level": "moderate"})
store.put(preferences_namespace, "schedule", {"preferred_workout_time": "morning", "available_days": ["Mon", "Wed", "Fri"]})

print("Stored Sarah's profile and preferences in long-term memory")

Stored Sarah's profile and preferences in long-term memory


In [14]:
# Retrieve specific memories
name = store.get(profile_namespace, "name")
print(f"Name: {name.value}")

goals = store.get(profile_namespace, "goals")
print(f"Goals: {goals.value}")

# List all memories in a namespace
print("\nAll profile items:")
for item in store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

Name: {'value': 'Sarah'}
Goals: {'primary': 'improve sleep', 'secondary': 'reduce stress'}

All profile items:
  name: {'value': 'Sarah'}
  goals: {'primary': 'improve sleep', 'secondary': 'reduce stress'}
  conditions: {'allergies': ['peanuts'], 'injuries': ['bad knee']}


In [16]:
from langgraph.store.base import BaseStore
from langchain_core.runnables import RunnableConfig

# Define state with user_id for personalization
class PersonalizedState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str


def personalized_wellness_chatbot(state: PersonalizedState, config: RunnableConfig, *, store: BaseStore):
    """A wellness chatbot that uses long-term memory for personalization."""
    user_id = state["user_id"]
    profile_namespace = (user_id, "profile")
    preferences_namespace = (user_id, "preferences")
    
    # Retrieve user profile from long-term memory
    profile_items = list(store.search(profile_namespace))
    pref_items = list(store.search(preferences_namespace))
    
    # Build context from profile
    profile_text = "\n".join([f"- {p.key}: {p.value}" for p in profile_items])
    pref_text = "\n".join([f"- {p.key}: {p.value}" for p in pref_items])
    
    system_msg = f"""You are a Personal Wellness Assistant. You know the following about this user:

PROFILE:
{profile_text if profile_text else 'No profile stored.'}

PREFERENCES:
{pref_text if pref_text else 'No preferences stored.'}

Use this information to personalize your responses. Be supportive and helpful."""
    
    messages = [SystemMessage(content=system_msg)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build the personalized graph
builder2 = StateGraph(PersonalizedState)
builder2.add_node("chatbot", personalized_wellness_chatbot)
builder2.add_edge(START, "chatbot")
builder2.add_edge("chatbot", END)

# Compile with BOTH checkpointer (short-term) AND store (long-term)
personalized_graph = builder2.compile(
    checkpointer=MemorySaver(),
    store=store
)

print("Personalized graph compiled with both short-term and long-term memory")

Personalized graph compiled with both short-term and long-term memory


In [17]:
# Test the personalized chatbot - it knows Sarah's profile!
config = {"configurable": {"thread_id": "personalized_thread_1"}}

response = personalized_graph.invoke(
    {
        "messages": [HumanMessage(content="What exercises would you recommend for me?")],
        "user_id": "user_sarah"
    },
    config
)

print("User: What exercises would you recommend for me?")
print(f"Assistant: {response['messages'][-1].content}")
print()
print("Notice: The agent knows about Sarah's bad knee without her mentioning it!")

User: What exercises would you recommend for me?
Assistant: Hi Sarah! It's great that you're looking to improve your sleep and reduce stress through exercise. Given your bad knee, we want to focus on low-impact activities that are gentle on your joints. Here are some exercises you might enjoy, especially in the morning:

1. **Walking**: A brisk walk is a fantastic way to start your day. It’s low-impact and can help clear your mind.

2. **Swimming**: If you have access to a pool, swimming is excellent for a full-body workout without stressing your knee.

3. **Cycling**: Riding a stationary bike or cycling outdoors can be a great way to get your heart rate up while being easy on your knees.

4. **Yoga**: Gentle yoga can help with flexibility, relaxation, and stress reduction. Look for classes that focus on restorative or yin yoga.

5. **Pilates**: This can strengthen your core and improve your overall body awareness without putting too much strain on your knee.

6. **Strength Training**:

In [18]:
# Even in a NEW thread, it still knows Sarah's profile
# because long-term memory is cross-thread!

new_config = {"configurable": {"thread_id": "personalized_thread_2"}}

response = personalized_graph.invoke(
    {
        "messages": [HumanMessage(content="Can you suggest a snack for me?")],
        "user_id": "user_sarah"
    },
    new_config
)

print("User (NEW thread): Can you suggest a snack for me?")
print(f"Assistant: {response['messages'][-1].content}")
print()
print("Notice: Even in a new thread, the agent knows Sarah has a peanut allergy!")

User (NEW thread): Can you suggest a snack for me?
Assistant: Absolutely, Sarah! Since you have a peanut allergy, let’s find something safe and delicious. How about some apple slices with almond butter? It’s a great combination of fiber and healthy fats, which can help keep you satisfied. Just make sure to check that the almond butter is made in a peanut-free facility. 

If you’re looking for something lighter, you could also try some Greek yogurt with a sprinkle of cinnamon or a handful of berries. Both options are nutritious and can be a nice treat! Let me know if you’d like more suggestions!

Notice: Even in a new thread, the agent knows Sarah has a peanut allergy!


## Task 5: Message Trimming & Context Management

Long conversations can exceed the LLM's context window. LangGraph provides utilities to manage message history:

- **`trim_messages`**: Keeps only recent messages up to a token limit
- **Summarization**: Compress older messages into summaries

### Why Trim Even with 128K Context?

Even with large context windows:
1. **Cost**: More tokens = higher API costs
2. **Latency**: Larger contexts take longer to process
3. **Quality**: Models can struggle with "lost in the middle" - important info buried in long contexts
4. **Relevance**: Old messages may not be relevant to current query

In [19]:
from langchain_core.messages import trim_messages

# Create a trimmer that keeps only recent messages
trimmer = trim_messages(
    max_tokens=500,  # Keep messages up to this token count
    strategy="last",  # Keep the most recent messages
    token_counter=llm,  # Use the LLM to count tokens
    include_system=True,  # Always keep system messages
    allow_partial=False,  # Don't cut messages in half
)

# Example: Create a long conversation
long_conversation = [
    SystemMessage(content="You are a wellness assistant."),
    HumanMessage(content="I want to improve my health."),
    AIMessage(content="Great goal! Let's start with exercise. What's your current activity level?"),
    HumanMessage(content="I walk about 30 minutes a day."),
    AIMessage(content="That's a good foundation. For cardiovascular health, aim for 150 minutes of moderate activity per week."),
    HumanMessage(content="What about nutrition?"),
    AIMessage(content="Focus on whole foods: vegetables, lean proteins, whole grains. Limit processed foods and added sugars."),
    HumanMessage(content="And sleep?"),
    AIMessage(content="Aim for 7-9 hours. Maintain a consistent sleep schedule and create a relaxing bedtime routine."),
    HumanMessage(content="What's the most important change I should make first?"),
]

# Trim to fit context window
trimmed = trimmer.invoke(long_conversation)
print(f"Original: {len(long_conversation)} messages")
print(f"Trimmed: {len(trimmed)} messages")
print("\nTrimmed conversation:")
for msg in trimmed:
    role = type(msg).__name__.replace("Message", "")
    content = msg.content[:60] + "..." if len(msg.content) > 60 else msg.content
    print(f"  {role}: {content}")

Original: 10 messages
Trimmed: 10 messages

Trimmed conversation:
  System: You are a wellness assistant.
  Human: I want to improve my health.
  AI: Great goal! Let's start with exercise. What's your current a...
  Human: I walk about 30 minutes a day.
  AI: That's a good foundation. For cardiovascular health, aim for...
  Human: What about nutrition?
  AI: Focus on whole foods: vegetables, lean proteins, whole grain...
  Human: And sleep?
  AI: Aim for 7-9 hours. Maintain a consistent sleep schedule and ...
  Human: What's the most important change I should make first?


In [20]:
# Summarization approach for longer conversations

def summarize_conversation(messages: list, max_messages: int = 6) -> list:
    """Summarize older messages to manage context length."""
    if len(messages) <= max_messages:
        return messages
    
    # Keep the system message and last few messages
    system_msg = messages[0] if isinstance(messages[0], SystemMessage) else None
    content_messages = messages[1:] if system_msg else messages
    
    if len(content_messages) <= max_messages:
        return messages
    
    old_messages = content_messages[:-max_messages+1]
    recent_messages = content_messages[-max_messages+1:]
    
    # Summarize old messages
    summary_prompt = f"""Summarize this conversation in 2-3 sentences, 
capturing key wellness topics discussed and any important user information:

{chr(10).join([f'{type(m).__name__}: {m.content[:200]}' for m in old_messages])}"""
    
    summary = llm.invoke(summary_prompt)
    
    # Return: system + summary + recent messages
    result = []
    if system_msg:
        result.append(system_msg)
    result.append(SystemMessage(content=f"[Previous conversation summary: {summary.content}]"))
    result.extend(recent_messages)
    
    return result


# Test summarization
summarized = summarize_conversation(long_conversation, max_messages=4)
print(f"Summarized: {len(summarized)} messages")
print("\nSummarized conversation:")
for msg in summarized:
    role = type(msg).__name__.replace("Message", "")
    content = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
    print(f"  {role}: {content}")

Summarized: 5 messages

Summarized conversation:
  System: You are a wellness assistant.
  System: [Previous conversation summary: The conversation centers around the user's desir...
  Human: And sleep?
  AI: Aim for 7-9 hours. Maintain a consistent sleep schedule and create a relaxing be...
  Human: What's the most important change I should make first?


---
## ❓ Question #1:

What are the trade-offs between **short-term memory** (checkpointer) vs **long-term memory** (store)? When should wellness data move from short-term to long-term?

Consider:
- What information should persist across sessions?
- What are the privacy implications of each?
- How would you decide what to promote from short-term to long-term?

##### Answer:
Short-term memory is automatic, lightweight, and scoped to a single conversation, making it ideal for maintaining context with minimal privacy risk, but it cannot support personalization across sessions. Long-term memory enables durable personalization and learning over time, but requires explicit design, careful scoping, and stronger privacy controls due to its persistent nature.


Wellness data should move to long-term memory only when it is stable, explicitly shared, and demonstrably useful for future interactions, such as recurring goals or chronic conditions, rather than transient conversational details.


## ❓ Question #2:

Why use message trimming with a 128K context window when HealthWellnessGuide.txt is only ~16KB? What should **always** be preserved when trimming a wellness consultation?

Consider:
- The "lost in the middle" phenomenon
- Cost and latency implications
- What user information is critical for safety (allergies, conditions, etc.)

##### Answer:
Even with a 128K context window, message trimming is still useful because sending unnecessary messages increases cost and response time, and important information can get lost in long conversations (“lost in the middle”). Keeping the context focused helps the model give clearer and more relevant wellness advice.

When trimming a wellness consultation, system instructions and safety-critical user information should always be preserved. This includes allergies, injuries, medical conditions, and current goals, since losing this context could lead to unsafe or inappropriate recommendations.

---
## 🏗️ Activity #1: Store & Retrieve User Wellness Profile

Build a complete wellness profile system that:
1. Defines a wellness profile schema (name, goals, conditions, preferences)
2. Creates functions to store and retrieve profile data
3. Builds a personalized wellness agent that uses the profile
4. Tests that different users get different advice

### Requirements:
- Define at least 5 profile attributes
- Support multiple users with different profiles
- Agent should reference profile data in responses

##### Step 1 — Define a wellness profile schema
In this step, I define a clear wellness profile “shape” that the agent can rely on. We use a `TypedDict` to keep the profile structured and easy to reason about, and I include at least five core attributes such as name, goals, conditions, allergies, and fitness level.


In [22]:
# Step 1: Define a wellness profile schema
# Example attributes: name, age, goals, conditions, allergies, fitness_level, preferred_activities
from typing import TypedDict, List

# Step 1: Define a wellness profile schema (>= 5 attributes)
class WellnessProfile(TypedDict, total=False):
    name: str
    age: int
    goals: List[str]
    conditions: List[str]
    allergies: List[str]
    fitness_level: str
    preferred_activities: List[str]
    sleep_goal_hours: float
    communication_style: str


##### Step 2 — Create helper functions to store and retrieve profiles

In this step, I create two small helper functions for long-term memory: one to store a user’s wellness profile and one to retrieve it later. I use a namespace `(user_id, "profile")` to keep each user’s data isolated and predictable across different conversation threads.


In [26]:
# Step 2: Create helper functions to store and retrieve profiles

def store_wellness_profile(store, user_id: str, profile: dict) -> None:
    """Store a user's wellness profile in long-term memory."""
    namespace = (user_id, "profile")
    # Store the full profile under a single key for simplicity
    store.put(namespace, "wellness_profile", profile)


def get_wellness_profile(store, user_id: str) -> dict:
    """Retrieve a user's wellness profile from long-term memory."""
    namespace = (user_id, "profile")
    item = store.get(namespace, "wellness_profile")
    return item.value if item else {}

print("Helper functions defined: store_wellness_profile(), get_wellness_profile()")



Helper functions defined: store_wellness_profile(), get_wellness_profile()


##### Step 3 — Create two different user profiles

In this step, I create two distinct wellness profiles for different users. Each profile contains multiple attributes and represents long-term information that should persist across sessions. These profiles will later be used to personalize the agent’s recommendations.


In [30]:
from langgraph.store.memory import InMemoryStore

# Step 3: Create two different user profiles

# Initialize the long-term memory store
store = InMemoryStore()

# User 1 profile
sarah_profile = {
    "name": "Ida",
    "age": 34,
    "goals": ["improve sleep", "reduce stress"],
    "conditions": ["bad knee"],
    "allergies": ["peanuts"],
    "fitness_level": "beginner",
    "preferred_activities": ["walking", "yoga"],
}

# User 2 profile
marko_profile = {
    "name": "Goran",
    "age": 41,
    "goals": ["build strength", "lose weight"],
    "conditions": ["lower back sensitivity"],
    "allergies": [],
    "fitness_level": "intermediate",
    "preferred_activities": ["strength training", "cycling"],
}

# Store profiles in long-term memory
store_wellness_profile(store, "user_ida", sarah_profile)
store_wellness_profile(store, "user_goran", marko_profile)

print("Stored wellness profiles for Ida and Goran.")

# Quick verification
print("Ida profile:", get_wellness_profile(store, "user_ida"))
print("Goran profile:", get_wellness_profile(store, "user_goran"))


Stored wellness profiles for Ida and Goran.
Ida profile: {'name': 'Ida', 'age': 34, 'goals': ['improve sleep', 'reduce stress'], 'conditions': ['bad knee'], 'allergies': ['peanuts'], 'fitness_level': 'beginner', 'preferred_activities': ['walking', 'yoga']}
Goran profile: {'name': 'Goran', 'age': 41, 'goals': ['build strength', 'lose weight'], 'conditions': ['lower back sensitivity'], 'allergies': [], 'fitness_level': 'intermediate', 'preferred_activities': ['strength training', 'cycling']}


##### Step 4 — Build a personalized agent that uses profiles

In this step, I build a wellness agent that reads the user’s long-term profile from the `InMemoryStore` and injects it into the system prompt. This allows the agent to personalize advice consistently across sessions while still using short-term memory (via `MemorySaver`) for the current conversation thread.


In [31]:
from typing import TypedDict, Annotated
from langchain_core.messages import SystemMessage
from langchain_core.runnables import RunnableConfig
from langgraph.store.base import BaseStore
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

# Step 4: Build a personalized agent that uses profiles

class PersonalizedState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str


def personalized_wellness_chatbot(state: PersonalizedState, config: RunnableConfig, *, store: BaseStore):
    """Wellness chatbot that personalizes responses using long-term profile data."""
    user_id = state["user_id"]
    profile = get_wellness_profile(store, user_id)

    name = profile.get("name", "the user")
    goals = profile.get("goals", [])
    conditions = profile.get("conditions", [])
    allergies = profile.get("allergies", [])
    fitness_level = profile.get("fitness_level", "unknown")
    preferred_activities = profile.get("preferred_activities", [])

    system_prompt = f"""
You are a friendly Personal Wellness Assistant.
Use the user's profile to personalize your advice.

USER PROFILE:
- name: {name}
- goals: {goals}
- fitness_level: {fitness_level}
- preferred_activities: {preferred_activities}
- conditions/injuries: {conditions}
- allergies: {allergies}

Safety: Avoid recommendations that conflict with injuries or allergies.
If something is unclear, ask one short clarifying question.
""".strip()

    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


builder = StateGraph(PersonalizedState)
builder.add_node("chatbot", personalized_wellness_chatbot)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

personalized_graph = builder.compile(
    checkpointer=MemorySaver(),  # short-term memory within a thread
    store=store                  # long-term memory across threads
)

print("Personalized wellness agent compiled successfully.")


Personalized wellness agent compiled successfully.


##### Step 5 — Test 1 with different users (profiles should change the advice)

In this step, I ask the same question for two different users. The agent should produce different recommendations by referencing each user’s stored profile (e.g., injuries, goals, and preferred activities). I also use different `thread_id` values to keep short-term memory separated per conversation.


In [32]:
from langchain_core.messages import HumanMessage

# Step 5: Test with different users - they should get different advice

question = "What exercises would you recommend for me this week?"

# Test user Ida
config_ida = {"configurable": {"thread_id": "thread_ida_1"}}
response_ida = personalized_graph.invoke(
    {
        "messages": [HumanMessage(content=question)],
        "user_id": "user_ida"
    },
    config_ida
)

print("=== Ida (user_ida) ===")
print(response_ida["messages"][-1].content)
print()

# Test user Goran
config_goran = {"configurable": {"thread_id": "thread_goran_1"}}
response_goran = personalized_graph.invoke(
    {
        "messages": [HumanMessage(content=question)],
        "user_id": "user_goran"
    },
    config_goran
)

print("=== Goran (user_goran) ===")
print(response_goran["messages"][-1].content)


=== Ida (user_ida) ===
Hi Ida! Since you're a beginner and have a bad knee, I recommend focusing on gentle activities that can help improve your sleep and reduce stress. Here are some exercises you can try this week:

1. **Walking**: Aim for 20-30 minutes of walking at a comfortable pace. You can do this in a park or around your neighborhood. Walking is great for reducing stress and improving your mood.

2. **Gentle Yoga**: Look for beginner yoga classes that focus on relaxation and stretching. Poses like Child's Pose, Cat-Cow, and Seated Forward Bend can be beneficial without putting too much strain on your knee.

3. **Chair Yoga**: If you're looking for something even gentler, chair yoga can be a great option. It allows you to perform yoga poses while seated, which can be easier on your knee.

4. **Breathing Exercises**: Incorporate some deep breathing or meditation sessions into your routine. This can help reduce stress and improve your overall well-being.

Make sure to listen to yo

##### Step 5 — Test 2 with the same question for different users

In this step, I ask the same wellness question to multiple users. Even though the input is identical, the agent should generate different recommendations by leveraging each user’s long-term wellness profile.


In [35]:
from langchain_core.messages import HumanMessage

# Step 5: Test with the SAME question for different users

question = (
    "I want to improve my overall health this week. "
    "What should I focus on in terms of exercise and recovery?"
)

# Test user Ida
config_ida = {"configurable": {"thread_id": "thread_ida_3"}}
response_ida = personalized_graph.invoke(
    {
        "messages": [HumanMessage(content=question)],
        "user_id": "user_ida"
    },
    config=config_ida
)

print("=== Ida (user_ida) ===")
print(response_ida["messages"][-1].content)
print()


# Test user Goran
config_goran = {"configurable": {"thread_id": "thread_goran_3"}}
response_goran = personalized_graph.invoke(
    {
        "messages": [HumanMessage(content=question)],
        "user_id": "user_goran"
    },
    config=config_goran
)

print("=== Goran (user_goran) ===")
print(response_goran["messages"][-1].content)


=== Ida (user_ida) ===
Hi Ida! To improve your overall health this week, let's focus on a balanced approach to exercise and recovery that aligns with your goals and preferences.

**Exercise:**
1. **Walking:** Aim for 30 minutes of walking most days of the week. You can break it into shorter sessions if needed. Walking is low-impact and great for your knee.
2. **Gentle Yoga:** Try to incorporate 2-3 sessions of beginner yoga. Focus on poses that promote relaxation and flexibility, avoiding any that put strain on your knee. Look for classes that emphasize restorative or gentle yoga.

**Recovery:**
1. **Sleep Routine:** Establish a calming bedtime routine. Consider activities like reading or gentle stretching before bed to signal to your body that it's time to wind down.
2. **Stress Management:** Incorporate mindfulness practices, such as deep breathing or meditation, for a few minutes each day. This can help reduce stress and improve your overall well-being.

Make sure to listen to your 

---
# 🤝 Breakout Room #2
## Advanced Memory & Integration

## Task 6: Semantic Memory (Embeddings + Search)

**Semantic memory** stores facts and retrieves them based on *meaning* rather than exact matches. This is like how you might remember "that restaurant with the great pasta" even if you can't remember its exact name.

In LangGraph, semantic memory uses:
- **Store with embeddings**: Converts text to vectors for similarity search
- **`store.search()`**: Finds relevant memories by semantic similarity

### How It Works

```
User asks: "What helps with headaches?"
         ↓
Query embedded → [0.2, 0.8, 0.1, ...]
         ↓
Compare with stored wellness facts:
  - "Hydration can relieve headaches" → 0.92 similarity ✓
  - "Exercise improves sleep" → 0.35 similarity
         ↓
Return: "Hydration can relieve headaches"
```

In [38]:
from langchain_openai import OpenAIEmbeddings

# Create embeddings model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create a store with semantic search enabled
semantic_store = InMemoryStore(
    index={
        "embed": embeddings,
        "dims": 1536,  # Dimension of text-embedding-3-small
    }
)

print("Semantic memory store created with embedding support")

Semantic memory store created with embedding support


In [39]:
# Store various wellness facts as semantic memories
namespace = ("wellness", "facts")

wellness_facts = [
    ("fact_1", {"text": "Drinking water can help relieve headaches caused by dehydration"}),
    ("fact_2", {"text": "Regular exercise improves sleep quality and helps you fall asleep faster"}),
    ("fact_3", {"text": "Deep breathing exercises can reduce stress and anxiety within minutes"}),
    ("fact_4", {"text": "Eating protein at breakfast helps maintain steady energy levels throughout the day"}),
    ("fact_5", {"text": "Blue light from screens can disrupt your circadian rhythm and sleep"}),
    ("fact_6", {"text": "Walking for 30 minutes daily can improve cardiovascular health"}),
    ("fact_7", {"text": "Magnesium-rich foods like nuts and leafy greens can help with muscle cramps"}),
    ("fact_8", {"text": "A consistent sleep schedule, even on weekends, improves overall sleep quality"}),
]

for key, value in wellness_facts:
    semantic_store.put(namespace, key, value)

print(f"Stored {len(wellness_facts)} wellness facts in semantic memory")

Stored 8 wellness facts in semantic memory


In [40]:
# Search semantically - notice we don't need exact matches!

queries = [
    "My head hurts, what should I do?",
    "How can I get better rest at night?",
    "I'm feeling stressed and anxious",
    "What should I eat in the morning?",
]

for query in queries:
    print(f"\nQuery: {query}")
    results = semantic_store.search(namespace, query=query, limit=2)
    for r in results:
        print(f"   {r.value['text']} (score: {r.score:.3f})")


Query: My head hurts, what should I do?
   Drinking water can help relieve headaches caused by dehydration (score: 0.327)
   Magnesium-rich foods like nuts and leafy greens can help with muscle cramps (score: 0.173)

Query: How can I get better rest at night?
   Regular exercise improves sleep quality and helps you fall asleep faster (score: 0.463)
   A consistent sleep schedule, even on weekends, improves overall sleep quality (score: 0.426)

Query: I'm feeling stressed and anxious
   Deep breathing exercises can reduce stress and anxiety within minutes (score: 0.415)
   Drinking water can help relieve headaches caused by dehydration (score: 0.224)

Query: What should I eat in the morning?
   Eating protein at breakfast helps maintain steady energy levels throughout the day (score: 0.467)
   Walking for 30 minutes daily can improve cardiovascular health (score: 0.249)


## Task 7: Building Semantic Wellness Knowledge Base

Let's load the HealthWellnessGuide.txt and create a semantic knowledge base that our agent can search.

This is similar to RAG from Session 4, but now using LangGraph's Store API instead of a separate vector database.

In [41]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load and chunk the wellness document
loader = TextLoader("data/HealthWellnessGuide.txt")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(documents)

print(f"Loaded and split into {len(chunks)} chunks")
print(f"\nSample chunk:\n{chunks[0].page_content[:200]}...")

Loaded and split into 45 chunks

Sample chunk:
The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND MOVEMENT

Chapter 1: Understanding Exercise Basics

Exercise is one of the most important things yo...


In [42]:
# Store chunks in semantic memory
knowledge_namespace = ("wellness", "knowledge")

for i, chunk in enumerate(chunks):
    semantic_store.put(
        knowledge_namespace,
        f"chunk_{i}",
        {"text": chunk.page_content, "source": "HealthWellnessGuide.txt"}
    )

print(f"Stored {len(chunks)} chunks in semantic knowledge base")

Stored 45 chunks in semantic knowledge base


In [43]:
# Build a semantic search wellness chatbot

class SemanticState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str


def semantic_wellness_chatbot(state: SemanticState, config: RunnableConfig, *, store: BaseStore):
    """A wellness chatbot that retrieves relevant facts using semantic search."""
    user_message = state["messages"][-1].content
    
    # Search for relevant knowledge
    knowledge_results = store.search(
        ("wellness", "knowledge"),
        query=user_message,
        limit=3
    )
    
    # Build context from retrieved knowledge
    if knowledge_results:
        knowledge_text = "\n\n".join([f"- {r.value['text']}" for r in knowledge_results])
        system_msg = f"""You are a Personal Wellness Assistant with access to a wellness knowledge base.

Relevant information from your knowledge base:
{knowledge_text}

Use this information to answer the user's question. If the information doesn't directly answer their question, use your general knowledge but mention what you found."""
    else:
        system_msg = "You are a Personal Wellness Assistant. Answer wellness questions helpfully."
    
    messages = [SystemMessage(content=system_msg)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build and compile
builder3 = StateGraph(SemanticState)
builder3.add_node("chatbot", semantic_wellness_chatbot)
builder3.add_edge(START, "chatbot")
builder3.add_edge("chatbot", END)

semantic_graph = builder3.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Semantic wellness chatbot ready")

Semantic wellness chatbot ready


In [44]:
# Test semantic retrieval
config = {"configurable": {"thread_id": "semantic_thread_1"}}

questions = [
    "What exercises can help with lower back pain?",
    "How can I improve my sleep quality?",
    "What should I eat for better gut health?",
]

for q in questions:
    response = semantic_graph.invoke(
        {"messages": [HumanMessage(content=q)], "user_id": "test_user"},
        config
    )
    print(f"\nUser: {q}")
    print(f"Assistant: {response['messages'][-1].content[:500]}...")


User: What exercises can help with lower back pain?
Assistant: There are several effective exercises that can help alleviate lower back pain. Here are some recommended ones:

1. **Cat-Cow Stretch**: Start on your hands and knees. Alternate between arching your back up (like a cat) and letting it sag down (like a cow). Perform 10-15 repetitions.

2. **Bird Dog**: From a hands and knees position, extend your opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.

3. **Partial Crunches**: Lie on...

User: How can I improve my sleep quality?
Assistant: Improving your sleep quality can be achieved through several essential sleep hygiene practices and creating an optimal sleep environment. Here are some tips:

### Sleep Hygiene Practices:
1. **Maintain a Consistent Sleep Schedule**: Go to bed and wake up at the same time every day, even on weekends.
2. **Create a Relaxing Bedtime Routine**: Engage in calming activities before

## Task 8: Episodic Memory (Few-Shot Learning)

**Episodic memory** stores past experiences and uses them as examples for future tasks. In humans, this is like remembering the first time you successfully gave someone good advice and using that experience to guide future interactions.

For AI agents, episodic memory is often implemented as **few-shot examples** - showing the model examples of correct behavior so it can learn patterns.

### The CoALA Framework Perspective

> "Facts can be written to semantic memory, whereas **experiences** can be written to episodic memory."

Key insight: Sometimes it's easier to **show** than **tell**. LLMs learn well from examples!

In [45]:
# Create episodic memory with past successful interactions
episodic_namespace = ("agent", "episodes")

successful_episodes = [
    {
        "situation": "User asked about starting exercise with joint pain",
        "input": "I want to start exercising but my knees hurt. What should I do?",
        "output": """For knee-friendly exercise, I'd recommend starting with low-impact activities:

1. **Swimming or water aerobics** - Water supports your joints while providing great cardio
2. **Cycling** - Either stationary or regular bikes are gentle on knees
3. **Elliptical machine** - Provides cardio without impact

Start slowly with 15-20 minutes and gradually increase. If you experience pain, stop and consult a healthcare provider. Would you like specific exercises for any of these?""",
        "feedback": "User appreciated the specific alternatives and safety advice"
    },
    {
        "situation": "User struggling with sleep and stress",
        "input": "I can't fall asleep because I'm always thinking about work. Help!",
        "output": """Racing thoughts at bedtime is common, especially with work stress. Here's a proven approach:

**Before bed:**
- Write down tomorrow's to-do list (gets thoughts out of your head)
- Set a "worry time" earlier in the evening - not at bedtime

**At bedtime:**
- Try the 4-7-8 breathing technique: Inhale 4 counts, hold 7, exhale 8
- Use the "cognitive shuffle" - think of random, unrelated objects (apple, guitar, mountain...)

Which of these would you like to try first?""",
        "feedback": "User found the structured approach and specific techniques helpful"
    },
    {
        "situation": "User asking about healthy eating on a budget",
        "input": "How can I eat healthy when I don't have much money?",
        "output": """Eating healthy on a budget is absolutely possible! Here are practical strategies:

**Budget-friendly nutritious foods:**
- Eggs, beans, lentils (cheap protein)
- Frozen vegetables (same nutrition as fresh, longer lasting)
- Oats, rice, whole grain bread
- Canned fish (tuna, salmon)

**Money-saving tips:**
- Buy in bulk when possible
- Plan meals around sales
- Cook in batches and freeze portions

What's your typical weekly food budget? I can help you create a specific meal plan.""",
        "feedback": "User valued the practical, actionable advice without judgment"
    },
]

for i, episode in enumerate(successful_episodes):
    semantic_store.put(
        episodic_namespace,
        f"episode_{i}",
        {
            "text": episode["situation"],  # Used for semantic search
            **episode
        }
    )

print(f"Stored {len(successful_episodes)} episodic memories (past successful interactions)")

Stored 3 episodic memories (past successful interactions)


In [46]:
class EpisodicState(TypedDict):
    messages: Annotated[list, add_messages]


def episodic_wellness_chatbot(state: EpisodicState, config: RunnableConfig, *, store: BaseStore):
    """A chatbot that learns from past successful interactions."""
    user_question = state["messages"][-1].content
    
    # Search for similar past experiences
    similar_episodes = store.search(
        ("agent", "episodes"),
        query=user_question,
        limit=1
    )
    
    # Build few-shot examples from past episodes
    if similar_episodes:
        episode = similar_episodes[0].value
        few_shot_example = f"""Here's an example of a similar wellness question I handled well:

User asked: {episode['input']}

My response was:
{episode['output']}

The user feedback was: {episode['feedback']}

Use this as inspiration for the style, structure, and tone of your response, but tailor it to the current question."""
        
        system_msg = f"""You are a Personal Wellness Assistant. Learn from your past successes:

{few_shot_example}"""
    else:
        system_msg = "You are a Personal Wellness Assistant. Be helpful, specific, and supportive."
    
    messages = [SystemMessage(content=system_msg)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build the episodic memory graph
builder4 = StateGraph(EpisodicState)
builder4.add_node("chatbot", episodic_wellness_chatbot)
builder4.add_edge(START, "chatbot")
builder4.add_edge("chatbot", END)

episodic_graph = builder4.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Episodic memory chatbot ready")

Episodic memory chatbot ready


In [47]:
# Test episodic memory - similar question to stored episode
config = {"configurable": {"thread_id": "episodic_thread_1"}}

response = episodic_graph.invoke(
    {"messages": [HumanMessage(content="I want to exercise more but I have a bad hip. What can I do?")]},
    config
)

print("User: I want to exercise more but I have a bad hip. What can I do?")
print(f"\nAssistant: {response['messages'][-1].content}")
print("\nNotice: The response structure mirrors the successful knee pain episode!")

User: I want to exercise more but I have a bad hip. What can I do?

Assistant: It's great that you want to exercise more! For a bad hip, focusing on low-impact activities can help you stay active while minimizing discomfort. Here are some options to consider:

1. **Swimming or water aerobics** - The buoyancy of water reduces stress on your hip joints while providing a full-body workout.
2. **Cycling** - Using a stationary bike or cycling outdoors can be gentle on your hips and still give you a good cardiovascular workout.
3. **Walking** - Start with short, flat walks and gradually increase your distance. Consider using supportive shoes and walking on softer surfaces like grass or tracks.
4. **Yoga or Pilates** - These practices can improve flexibility and strength without putting too much strain on your hips. Look for classes that focus on gentle movements.
5. **Resistance training** - Using resistance bands or light weights can help strengthen the muscles around your hip, providing be

## Task 9: Procedural Memory (Self-Improving Agent)

**Procedural memory** stores the rules and instructions that guide behavior. In humans, this is like knowing *how* to give good advice - it's internalized knowledge about performing tasks.

For AI agents, procedural memory often means **self-modifying prompts**. The agent can:
1. Store its current instructions in the memory store
2. Reflect on feedback from interactions
3. Update its own instructions to improve

### The Reflection Pattern

```
User feedback: "Your advice is too long and complicated"
         ↓
Agent reflects on current instructions
         ↓
Agent updates instructions: "Keep advice concise and actionable"
         ↓
Future responses use updated instructions
```

In [48]:
# Initialize procedural memory with base instructions
procedural_namespace = ("agent", "instructions")

initial_instructions = """You are a Personal Wellness Assistant.

Guidelines:
- Be supportive and non-judgmental
- Provide evidence-based wellness information
- Ask clarifying questions when needed
- Encourage healthy habits without being preachy"""

semantic_store.put(
    procedural_namespace,
    "wellness_assistant",
    {"instructions": initial_instructions, "version": 1}
)

print("Initialized procedural memory with base instructions")
print(f"\nCurrent Instructions (v1):\n{initial_instructions}")

Initialized procedural memory with base instructions

Current Instructions (v1):
You are a Personal Wellness Assistant.

Guidelines:
- Be supportive and non-judgmental
- Provide evidence-based wellness information
- Ask clarifying questions when needed
- Encourage healthy habits without being preachy


In [49]:
class ProceduralState(TypedDict):
    messages: Annotated[list, add_messages]
    feedback: str  # Optional feedback from user


def get_instructions(store: BaseStore) -> tuple[str, int]:
    """Retrieve current instructions from procedural memory."""
    item = store.get(("agent", "instructions"), "wellness_assistant")
    if item is None:
        return "You are a helpful wellness assistant.", 0
    return item.value["instructions"], item.value["version"]


def procedural_assistant_node(state: ProceduralState, config: RunnableConfig, *, store: BaseStore):
    """Respond using current procedural instructions."""
    instructions, version = get_instructions(store)
    
    messages = [SystemMessage(content=instructions)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


def reflection_node(state: ProceduralState, config: RunnableConfig, *, store: BaseStore):
    """Reflect on feedback and update instructions if needed."""
    feedback = state.get("feedback", "")
    
    if not feedback:
        return {}  # No feedback, no update needed
    
    # Get current instructions
    current_instructions, version = get_instructions(store)
    
    # Ask the LLM to reflect and improve instructions
    reflection_prompt = f"""You are improving a wellness assistant's instructions based on user feedback.

Current Instructions:
{current_instructions}

User Feedback:
{feedback}

Based on this feedback, provide improved instructions. Keep the same general format but incorporate the feedback.
Only output the new instructions, nothing else."""
    
    response = llm.invoke([HumanMessage(content=reflection_prompt)])
    new_instructions = response.content
    
    # Update procedural memory with new instructions
    store.put(
        ("agent", "instructions"),
        "wellness_assistant",
        {"instructions": new_instructions, "version": version + 1}
    )
    
    print(f"\nInstructions updated to version {version + 1}")
    return {}


def should_reflect(state: ProceduralState) -> str:
    """Decide whether to reflect on feedback."""
    if state.get("feedback"):
        return "reflect"
    return "end"


# Build the procedural memory graph
builder5 = StateGraph(ProceduralState)
builder5.add_node("assistant", procedural_assistant_node)
builder5.add_node("reflect", reflection_node)

builder5.add_edge(START, "assistant")
builder5.add_conditional_edges("assistant", should_reflect, {"reflect": "reflect", "end": END})
builder5.add_edge("reflect", END)

procedural_graph = builder5.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Procedural memory graph ready (with self-improvement capability)")

Procedural memory graph ready (with self-improvement capability)


In [50]:
# Test with initial instructions
config = {"configurable": {"thread_id": "procedural_thread_1"}}

response = procedural_graph.invoke(
    {
        "messages": [HumanMessage(content="How can I reduce stress?")],
        "feedback": ""  # No feedback yet
    },
    config
)

print("User: How can I reduce stress?")
print(f"\nAssistant (v1 instructions):\n{response['messages'][-1].content}")

User: How can I reduce stress?

Assistant (v1 instructions):
Reducing stress is a great goal, and there are several effective strategies you can try. Here are some evidence-based methods:

1. **Mindfulness and Meditation**: Practicing mindfulness or meditation can help you stay present and reduce anxiety. Even a few minutes a day can make a difference.

2. **Physical Activity**: Regular exercise is a powerful stress reliever. It can boost your mood and improve your overall well-being. Find an activity you enjoy, whether it's walking, dancing, or yoga.

3. **Deep Breathing**: Simple deep breathing exercises can help calm your mind and body. Try inhaling deeply through your nose, holding for a few seconds, and exhaling slowly through your mouth.

4. **Connect with Others**: Talking to friends or family can provide support and help you feel less isolated. Sometimes just sharing your feelings can lighten the load.

5. **Time Management**: Organizing your tasks and setting priorities can he

In [51]:
# Now provide feedback - the agent will update its own instructions!
response = procedural_graph.invoke(
    {
        "messages": [HumanMessage(content="How can I reduce stress?")],
        "feedback": "Your responses are too long. Please be more concise and give me 3 actionable tips maximum."
    },
    {"configurable": {"thread_id": "procedural_thread_2"}}
)


Instructions updated to version 2


In [52]:
# Check the updated instructions
new_instructions, version = get_instructions(semantic_store)
print(f"Updated Instructions (v{version}):\n")
print(new_instructions)

Updated Instructions (v2):

You are a Personal Wellness Assistant.

Guidelines:
- Be supportive and non-judgmental
- Provide concise, evidence-based wellness information
- Offer a maximum of 3 actionable tips
- Ask clarifying questions when needed
- Encourage healthy habits without being preachy


In [53]:
# Test with updated instructions - should be more concise now!
response = procedural_graph.invoke(
    {
        "messages": [HumanMessage(content="How can I sleep better?")],
        "feedback": ""  # No feedback this time
    },
    {"configurable": {"thread_id": "procedural_thread_3"}}
)

print(f"User: How can I sleep better?")
print(f"\nAssistant (v{version} instructions - after feedback):")
print(response['messages'][-1].content)
print("\nNotice: The response should now be more concise based on the feedback!")

User: How can I sleep better?

Assistant (v2 instructions - after feedback):
Improving your sleep can have a significant impact on your overall wellness. Here are three actionable tips to help you sleep better:

1. **Establish a Consistent Sleep Schedule**: Try to go to bed and wake up at the same time every day, even on weekends. This helps regulate your body's internal clock.

2. **Create a Relaxing Bedtime Routine**: Engage in calming activities before bed, such as reading, taking a warm bath, or practicing relaxation techniques like deep breathing or meditation. This signals to your body that it's time to wind down.

3. **Limit Screen Time Before Bed**: Reduce exposure to screens (phones, tablets, TVs) at least an hour before bedtime. The blue light emitted can interfere with your ability to fall asleep.

Are there specific challenges you're facing with your sleep that you'd like to discuss?

Notice: The response should now be more concise based on the feedback!


## Task 10: Unified Wellness Memory Agent

Now let's combine **all 5 memory types** into a unified wellness agent:

1. **Short-term**: Remembers current conversation (checkpointer)
2. **Long-term**: Stores user profile across sessions (store + namespace)
3. **Semantic**: Retrieves relevant wellness knowledge (store + embeddings)
4. **Episodic**: Uses past successful interactions as examples (store + search)
5. **Procedural**: Adapts behavior based on feedback (store + reflection)

### Memory Retrieval Flow

```
User Query: "What exercises can help my back pain?"
              │
              ▼
┌─────────────────────────────────────────────────┐
│  1. PROCEDURAL: Get current instructions         │
│  2. LONG-TERM: Load user profile (conditions)    │
│  3. SEMANTIC: Search wellness knowledge          │
│  4. EPISODIC: Find similar past interactions     │
│  5. SHORT-TERM: Include conversation history     │
└─────────────────────────────────────────────────┘
              │
              ▼
        Generate personalized, informed response
```

In [54]:
class UnifiedState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str
    feedback: str


def unified_wellness_assistant(state: UnifiedState, config: RunnableConfig, *, store: BaseStore):
    """An assistant that uses all five memory types."""
    user_id = state["user_id"]
    user_message = state["messages"][-1].content
    
    # 1. PROCEDURAL: Get current instructions
    instructions_item = store.get(("agent", "instructions"), "wellness_assistant")
    base_instructions = instructions_item.value["instructions"] if instructions_item else "You are a helpful wellness assistant."
    
    # 2. LONG-TERM: Get user profile
    profile_items = list(store.search((user_id, "profile")))
    pref_items = list(store.search((user_id, "preferences")))
    profile_text = "\n".join([f"- {p.key}: {p.value}" for p in profile_items]) if profile_items else "No profile stored."
    
    # 3. SEMANTIC: Search for relevant knowledge
    relevant_knowledge = store.search(("wellness", "knowledge"), query=user_message, limit=2)
    knowledge_text = "\n".join([f"- {r.value['text'][:200]}..." for r in relevant_knowledge]) if relevant_knowledge else "No specific knowledge found."
    
    # 4. EPISODIC: Find similar past interactions
    similar_episodes = store.search(("agent", "episodes"), query=user_message, limit=1)
    if similar_episodes:
        ep = similar_episodes[0].value
        episode_text = f"Similar past interaction:\nUser: {ep.get('input', 'N/A')}\nResponse style: {ep.get('feedback', 'N/A')}"
    else:
        episode_text = "No similar past interactions found."
    
    # Build comprehensive system message
    system_message = f"""{base_instructions}

=== USER PROFILE ===
{profile_text}

=== RELEVANT WELLNESS KNOWLEDGE ===
{knowledge_text}

=== LEARNING FROM EXPERIENCE ===
{episode_text}

Use all of this context to provide the best possible personalized response."""
    
    # 5. SHORT-TERM: Full conversation history is automatically managed by the checkpointer
    # Use summarization for long conversations
    trimmed_messages = summarize_conversation(state["messages"], max_messages=6)
    
    messages = [SystemMessage(content=system_message)] + trimmed_messages
    response = llm.invoke(messages)
    return {"messages": [response]}


def unified_feedback_node(state: UnifiedState, config: RunnableConfig, *, store: BaseStore):
    """Update procedural memory based on feedback."""
    feedback = state.get("feedback", "")
    if not feedback:
        return {}
    
    item = store.get(("agent", "instructions"), "wellness_assistant")
    if item is None:
        return {}
    
    current = item.value
    reflection_prompt = f"""Update these instructions based on feedback:

Current: {current['instructions']}
Feedback: {feedback}

Output only the updated instructions."""
    
    response = llm.invoke([HumanMessage(content=reflection_prompt)])
    store.put(
        ("agent", "instructions"),
        "wellness_assistant",
        {"instructions": response.content, "version": current["version"] + 1}
    )
    print(f"Procedural memory updated to v{current['version'] + 1}")
    return {}


def unified_route(state: UnifiedState) -> str:
    return "feedback" if state.get("feedback") else "end"


# Build the unified graph
unified_builder = StateGraph(UnifiedState)
unified_builder.add_node("assistant", unified_wellness_assistant)
unified_builder.add_node("feedback", unified_feedback_node)

unified_builder.add_edge(START, "assistant")
unified_builder.add_conditional_edges("assistant", unified_route, {"feedback": "feedback", "end": END})
unified_builder.add_edge("feedback", END)

# Compile with both checkpointer (short-term) and store (all other memory types)
unified_graph = unified_builder.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Unified wellness assistant ready with all 5 memory types!")

Unified wellness assistant ready with all 5 memory types!


In [55]:
# Test the unified assistant
config = {"configurable": {"thread_id": "unified_thread_1"}}

# First interaction - should use semantic + long-term + episodic memory
response = unified_graph.invoke(
    {
        "messages": [HumanMessage(content="What exercises would you recommend for my back?")],
        "user_id": "user_sarah",  # Sarah has a bad knee in her profile!
        "feedback": ""
    },
    config
)

print("User: What exercises would you recommend for my back?")
print(f"\nAssistant: {response['messages'][-1].content}")
print("\n" + "="*60)
print("Memory types used:")
print("  Long-term: Knows Sarah has a bad knee")
print("  Semantic: Retrieved back exercise info from knowledge base")
print("  Episodic: May use similar joint pain episode as reference")
print("  Procedural: Following current instructions")
print("  Short-term: Will remember this in follow-up questions")

User: What exercises would you recommend for my back?

Assistant: It's great that you're looking to support your back health! Here are three gentle exercises that can help relieve lower back pain:

1. **Cat-Cow Stretch**: Start on your hands and knees. Alternate between arching your back up (like a cat) and letting it sag down (like a cow). Repeat this for 10-15 repetitions. This helps improve flexibility and relieve tension.

2. **Child's Pose**: Kneel on the floor, sit back on your heels, and stretch your arms forward on the ground. Hold this position for 20-30 seconds. It gently stretches the back and can provide relief.

3. **Pelvic Tilts**: Lie on your back with your knees bent and feet flat on the floor. Gently flatten your lower back against the floor by tilting your pelvis upward. Hold for a few seconds, then relax. Repeat this 10-15 times.

How does your back feel currently? Are there specific areas of discomfort you’d like to focus on?

Memory types used:
  Long-term: Knows S

In [56]:
# Follow-up question (tests short-term memory)
response = unified_graph.invoke(
    {
        "messages": [HumanMessage(content="Can you show me how to do the first one?")],
        "user_id": "user_sarah",
        "feedback": ""
    },
    config  # Same thread
)

print("User: Can you show me how to do the first one?")
print(f"\nAssistant: {response['messages'][-1].content}")
print("\nNotice: The agent remembers the context from the previous message!")

User: Can you show me how to do the first one?

Assistant: I can't physically show you, but I can definitely guide you through the Cat-Cow Stretch step-by-step:

1. **Start Position**: Get on your hands and knees on a comfortable surface, like a yoga mat. Your wrists should be directly under your shoulders, and your knees should be under your hips.

2. **Cat Position**: Inhale deeply. As you exhale, round your back towards the ceiling, tucking your chin to your chest. Imagine you're trying to create a "C" shape with your spine. Hold this position for a moment.

3. **Cow Position**: Inhale again. As you exhale, arch your back, letting your belly drop towards the floor. Lift your head and tailbone towards the ceiling, creating a gentle curve in your spine.

4. **Repeat**: Continue to alternate between these two positions for 10-15 repetitions, synchronizing your breath with your movements.

Make sure to move slowly and listen to your body. If you feel any discomfort, ease out of the stre

---
## ❓ Question #3:

How would you decide what constitutes a **"successful" wellness interaction** worth storing as an episode? What metadata should you store alongside the episode?

Consider:
- Explicit feedback (thumbs up) vs implicit signals
- User engagement (did they ask follow-up questions?)
- Objective outcomes vs subjective satisfaction
- Privacy implications of storing interaction data

##### Answer:
For me, a “successful” wellness interaction is one where the user not only gets correct and safe advice, but also feels understood and motivated to act on it. Explicit signals like positive feedback or a thumbs-up are the clearest indicators, but I’d also treat continued engagement (follow-up questions, asking for a plan, or clarifying details) as a strong sign that the advice resonated.

When storing an episode, I would keep just enough metadata to reuse the pattern effectively: the user’s goal or situation, the original question, the structure and tone of the response, and a brief success signal (e.g., “user engaged further” or “feedback was positive”). I would avoid storing unnecessary personal or sensitive data, focusing instead on how the advice worked rather than who the user was, to balance learning value with privacy.

## ❓ Question #4:

For a **production wellness assistant**, which memory types need persistent storage (PostgreSQL) vs in-memory? How would you handle memory across multiple agent instances (e.g., Exercise Agent, Nutrition Agent, Sleep Agent)?

Consider:
- Which memories are user-specific vs shared?
- Consistency requirements across agents
- Memory expiration and cleanup policies
- Namespace strategy for multi-agent systems

##### Answer:
In a production wellness assistant, information that needs to be remembered over time, like user profiles, preferences, and agent instructions, should be stored in persistent storage such as PostgreSQL. Short-term conversation memory can stay in-memory because it is only needed during a single interaction and can be safely discarded afterward.

When using multiple agents (Exercise, Nutrition, Sleep), user-related memory should be shared so all agents understand the same user, while each agent can have its own domain knowledge. Using clear namespaces and cleaning up old or unused memories helps keep the system consistent, scalable, and respectful of user privacy.

---
## 🏗️ Activity #2: Wellness Memory Dashboard

Build a wellness tracking system that:
1. Tracks wellness metrics over time (mood, energy, sleep quality)
2. Uses semantic memory to find relevant advice
3. Uses episodic memory to recall what worked before
4. Uses procedural memory to adapt advice style
5. Provides a synthesized "wellness summary"

### Requirements:
- Store at least 3 wellness metrics per user
- Track metrics over multiple "days" (simulated)
- Agent should reference historical data in responses
- Generate a personalized wellness summary

##### Step 1 — Define a personal wellness metrics schema 
In this step, I set up a simple weekly wellness dashboard for myself. I track three core daily metrics — `sleep_quality`, `energy`, and `mood` (rated 1–10) — and optionally include fitness signals like `workout_minutes` or `steps`. These metrics are stored in a per-user namespace so the agent can analyze trends over time and provide personalized wellness recommendations.



In [85]:
### YOUR CODE HERE ###
# Step 1: Define wellness metrics schema and storage functions
from datetime import datetime, timedelta

# Core metrics for my personal wellness dashboard
CORE_METRICS = {"sleep_quality", "energy", "mood"}
OPTIONAL_METRICS = {"workout_minutes", "steps"}

def log_wellness_metric(
    store,
    user_id: str,
    date: str,
    metric_type: str,
    value: float,
    notes: str = ""
):
    """Log a wellness metric for a user."""
    namespace = (user_id, "metrics")

    # Basic validation to keep data clean
    if metric_type not in (CORE_METRICS | OPTIONAL_METRICS):
        raise ValueError(
            f"Unknown metric_type '{metric_type}'. "
            f"Use one of {sorted(CORE_METRICS | OPTIONAL_METRICS)}"
        )

    key = f"{date}:{metric_type}"

    store.put(
        namespace,
        key,
        {
            "date": date,                 # YYYY-MM-DD
            "metric_type": metric_type,
            "value": float(value),        # 1–10 for core metrics
            "notes": notes,
        }
    )

def get_wellness_history(store, user_id: str, metric_type: str = None, days: int = 7) -> list:
    namespace = (user_id, "metrics")

    today = datetime.now().date()
    cutoff_date = today - timedelta(days=days - 1)

    items = list(store.search(namespace, limit=1000))  

    history = []
    for item in items:
        record = item.value
        try:
            record_date = datetime.fromisoformat(record["date"]).date()
        except Exception:
            continue

        if record_date < cutoff_date:
            continue
        if metric_type and record["metric_type"] != metric_type:
            continue

        history.append(record)

    history.sort(key=lambda x: (x["date"], x["metric_type"]))
    return history


print("Step 1 complete: wellness metric logging and history retrieval ready.")

Step 1 complete: wellness metric logging and history retrieval ready.


In [86]:
namespace = ("user_goran", "metrics")

all_items = list(dashboard_store.search(namespace, limit=1000))
all_days = sorted({it.value["date"] for it in all_items})

print("Days found:", all_days)
print("Number of days:", len(all_days))
print("Total records:", len(all_items))


Days found: ['2026-01-25', '2026-01-26', '2026-01-27', '2026-01-28', '2026-01-29', '2026-01-30', '2026-01-31']
Number of days: 7
Total records: 28


##### Step 2 — Create sample wellness data for a user (simulate a week)

In this step, I simulate one week of wellness tracking for my user profile. I log daily values for `sleep_quality`, `energy`, and `mood` (1–10), and I also include a fitness signal (`workout_minutes`) to make the weekly trend more realistic. This data will be used later to generate a personalized wellness summary and recommendations.


In [87]:
# Step 2: Create sample wellness data for a user (simulate a week)

from datetime import date, timedelta
from langgraph.store.memory import InMemoryStore


# Use an in-memory store for the dashboard metrics (can also reuse semantic_store if you prefer)
dashboard_store = InMemoryStore()

user_id = "user_goran"

# Simulate the last 7 days 
start_day = date.today() - timedelta(days=6)
days = [start_day + timedelta(days=i) for i in range(7)]

# Sample week: a couple of lower sleep days -> lower energy/mood
sample_week = [
    # date, sleep_quality, energy, mood, workout_minutes, notes
    (days[0], 7, 7, 7, 35, "Light workout, felt good overall"),
    (days[1], 6, 6, 6,  0, "Busy day, less movement"),
    (days[2], 5, 5, 5, 20, "Short session, poor sleep"),
    (days[3], 6, 6, 6, 40, "Better routine, moderate training"),
    (days[4], 7, 7, 7,  0, "Rest day, solid sleep"),
    (days[5], 8, 8, 8, 45, "Great energy, strong workout"),
    (days[6], 7, 7, 7, 25, "Good day, lighter session"),
]

for d, sleep_q, energy, mood, workout_min, notes in sample_week:
    iso = d.isoformat()
    log_wellness_metric(dashboard_store, user_id, iso, "sleep_quality", sleep_q, notes=notes)
    log_wellness_metric(dashboard_store, user_id, iso, "energy", energy, notes=notes)
    log_wellness_metric(dashboard_store, user_id, iso, "mood", mood, notes=notes)
    log_wellness_metric(dashboard_store, user_id, iso, "workout_minutes", workout_min, notes=notes)

print("Step 2 complete: logged a simulated week of wellness metrics.")


Step 2 complete: logged a simulated week of wellness metrics.


##### Step 3 — Build a wellness dashboard agent

In this step, I build a wellness dashboard agent that analyzes my tracked metrics over time and turns them into meaningful insights. The agent retrieves my recent wellness history, looks for simple patterns (such as low sleep leading to low energy), searches for relevant wellness advice using semantic memory, recalls what has worked well before using episodic memory, and finally generates a personalized weekly wellness summary with actionable recommendations.


In [89]:
from typing import TypedDict, Annotated
from collections import defaultdict
from statistics import mean

from langchain_core.messages import SystemMessage
from langchain_core.runnables import RunnableConfig
from langgraph.store.base import BaseStore
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

# Step 3: Build a wellness dashboard agent that:
#   - Retrieves user's wellness history
#   - Searches for relevant advice based on patterns
#   - Uses episodic memory for what worked before
#   - Generates a personalized summary

class DashboardState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str
    days: int


def _group_by_day(history: list) -> dict:
    """Group flat metric records into a per-day structure."""
    by_day = defaultdict(dict)
    for r in history:
        by_day[r["date"]][r["metric_type"]] = r["value"]
    return dict(by_day)


def _analyze_patterns(by_day: dict) -> dict:
    """Detect simple weekly patterns from wellness metrics."""
    stats = {}
    flags = []

    for metric in ["sleep_quality", "energy", "mood"]:
        values = [v.get(metric) for v in by_day.values() if metric in v]
        if values:
            stats[metric] = {
                "avg": round(mean(values), 1),
                "min": min(values),
                "max": max(values),
            }

    if stats.get("sleep_quality") and stats["sleep_quality"]["avg"] < 6:
        flags.append("Average sleep quality was lower than ideal this week.")
    if stats.get("energy") and stats["energy"]["avg"] < 6:
        flags.append("Energy levels were consistently low during the week.")
    if stats.get("mood") and stats["mood"]["avg"] < 6:
        flags.append("Mood scores suggest increased stress or fatigue.")

    return {"stats": stats, "flags": flags}


def dashboard_agent_node(state: DashboardState, config: RunnableConfig, *, store: BaseStore):
    user_id = state["user_id"]
    days = state.get("days", 7)

    # 1) Retrieve user's wellness history
    history = get_wellness_history(dashboard_store, user_id, days=days)
    by_day = _group_by_day(history)

    # 2) Detect patterns in the data
    patterns = _analyze_patterns(by_day)

    # Build a compact dashboard view
    dashboard_lines = []
    for day, metrics in sorted(by_day.items()):
        dashboard_lines.append(
            f"{day}: sleep={metrics.get('sleep_quality','?')}, "
            f"energy={metrics.get('energy','?')}, "
            f"mood={metrics.get('mood','?')}, "
            f"workout_min={metrics.get('workout_minutes','?')}"
        )
    dashboard_text = "\n".join(dashboard_lines) if dashboard_lines else "No recent data available."

    # 3) Search for relevant advice based on detected patterns
    semantic_query = f"Wellness patterns: {patterns['flags']}"
    advice_hits = store.search(
        ("wellness", "knowledge"),
        query=semantic_query,
        limit=3
    )
    advice_text = "\n".join([f"- {h.value['text'][:400]}" for h in advice_hits])

    # 4) Use episodic memory to recall what worked before
    episode_hits = store.search(
        ("agent", "episodes"),
        query=semantic_query,
        limit=1
    )
    episode_block = ""
    if episode_hits:
        ep = episode_hits[0].value
        episode_block = f"""
Past successful approach:
User: {ep.get('input')}
Assistant: {ep.get('output')}
""".strip()

    # 5) Generate a personalized summary
    instructions_item = store.get(("agent", "instructions"), "wellness_assistant")
    base_instructions = instructions_item.value["instructions"] if instructions_item else "You are a helpful wellness assistant."

    system_prompt = f"""
{base_instructions}

You are generating a personalized weekly wellness dashboard for me.
Use my tracked metrics to summarize trends, explain likely causes, and suggest clear next steps.

WELLNESS DATA (last {days} days):
{dashboard_text}

DETECTED PATTERNS:
{patterns['flags'] if patterns['flags'] else "No major negative patterns detected."}

RELEVANT ADVICE (semantic memory):
{advice_text}

{episode_block}

Output requirements:
- Start with a short weekly summary (2-3 sentences).
- Then list up to 4 key insights as bullet points.
- Finish with a numbered action plan (max 5 items, no blank lines).
""".strip()

    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Compile the dashboard graph
builder = StateGraph(DashboardState)
builder.add_node("dashboard", dashboard_agent_node)
builder.add_edge(START, "dashboard")
builder.add_edge("dashboard", END)

dashboard_graph = builder.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Step 3 complete: wellness dashboard agent compiled.")



Step 3 complete: wellness dashboard agent compiled.


##### Step 4 — Test the wellness dashboard

In this step, I test the wellness dashboard agent with natural-language questions.  
The goal is to verify that the agent correctly references my recent wellness history, identifies patterns, pulls relevant advice, and produces a clear, personalized summary.


In [92]:
from langchain_core.messages import HumanMessage

# Step 4: Test the dashboard
# Example: "Give me a summary of my wellness this week"
# Example: "I've been feeling tired lately. What might help?"

config = {"configurable": {"thread_id": "dashboard_test_1"}}

questions = [
    "Give me a summary of my wellness this week.",
    "I've been feeling tired lately. What might help?",
    "My energy feels inconsistent this week. Based on my data, what should I focus on improving first?"
]

for q in questions:
    result = dashboard_graph.invoke(
        {
            "messages": [HumanMessage(content=q)],
            "user_id": "user_goran",
            "days": 7
        },
        config=config
    )
    print(f"\nUser: {q}")
    print("Assistant:")
    print(result["messages"][-1].content)



User: Give me a summary of my wellness this week.
Assistant:
This week, your wellness metrics show a mix of positive and challenging trends, particularly in sleep and energy levels. While you had some days with good sleep and physical activity, there were also days where both sleep and energy dipped, contributing to feelings of tiredness.

**Key Insights:**
- Your average sleep duration was around 6.4 hours, which is slightly below the recommended 7-9 hours for optimal health.
- Energy and mood levels fluctuated, with lower scores on days with less sleep and no workouts, indicating a potential link between these factors.
- You engaged in physical activity on most days, with a peak of 45 minutes, which positively correlates with your higher energy and mood scores.
- Consistency in sleep and exercise could enhance your overall wellness and help stabilize your energy levels.

**Action Plan:**
1. Aim for at least 7 hours of sleep each night by establishing a consistent bedtime routine.
2.

In [93]:
from collections import defaultdict

history = get_wellness_history(dashboard_store, "user_goran", days=7)
by_day = defaultdict(dict)

for r in history:
    by_day[r["date"]][r["metric_type"]] = r["value"]

print("Date       sleep  energy  mood  workout_min")
for d in sorted(by_day.keys()):
    row = by_day[d]
    print(f"{d}   {row.get('sleep_quality','-'):>5}  {row.get('energy','-'):>6}  {row.get('mood','-'):>4}  {row.get('workout_minutes','-'):>11}")


Date       sleep  energy  mood  workout_min
2026-01-25     7.0     7.0   7.0         35.0
2026-01-26     6.0     6.0   6.0          0.0
2026-01-27     5.0     5.0   5.0         20.0
2026-01-28     6.0     6.0   6.0         40.0
2026-01-29     7.0     7.0   7.0          0.0
2026-01-30     8.0     8.0   8.0         45.0
2026-01-31     7.0     7.0   7.0         25.0


---
## Summary

In this session, we explored the **5 memory types** from the CoALA framework:

| Memory Type | LangGraph Component | Scope | Wellness Use Case |
|-------------|---------------------|-------|-------------------|
| **Short-term** | `MemorySaver` + `thread_id` | Within thread | Current consultation |
| **Long-term** | `InMemoryStore` + namespaces | Across threads | User profile, goals |
| **Semantic** | Store + embeddings + `search()` | Across threads | Knowledge retrieval |
| **Episodic** | Store + few-shot examples | Across threads | Past successful interactions |
| **Procedural** | Store + self-reflection | Across threads | Self-improving instructions |

### Key Takeaways:

1. **Memory transforms chatbots into assistants** - Persistence enables personalization
2. **Different memory types serve different purposes** - Choose based on your use case
3. **Context management is critical** - Trim and summarize to stay within limits
4. **Episodic memory enables learning** - Show, don't just tell
5. **Procedural memory enables adaptation** - Agents can improve themselves

### Production Considerations:

- Use `PostgresSaver` instead of `MemorySaver` for persistent checkpoints
- Use `PostgresStore` instead of `InMemoryStore` for persistent long-term memory
- Consider TTL (Time-to-Live) policies for automatic memory cleanup
- Implement proper access controls for user data

### Further Reading:

- [LangGraph Memory Documentation](https://langchain-ai.github.io/langgraph/concepts/memory/)
- [CoALA Paper](https://arxiv.org/abs/2309.02427) - Cognitive Architectures for Language Agents
- [LangGraph Platform](https://docs.langchain.com/langgraph-platform/) - Managed infrastructure for production